In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
# Datasetの取得 https://www.tensorflow.org/tutorials/load_data/images?hl=ja
'''
import tensorflow as tf

AUTOTUNE = tf.data.experimental.AUTOTUNE

import pathlib
data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)
'''

In [2]:
# dataフォルダ内の各花の写真が保存されたフォルダリストと、フォルダ名からラベルを取得

import pathlib
data_folder = pathlib.Path('data')
flower_folders = []
Labels = []
for item in data_folder.iterdir():
    flower_folders.append(item)
    # フォルダ名からラベルを取得
    Labels.append(str(item).split('\\')[1])

print(flower_folders)
print(Labels)

[WindowsPath('data/daisy'), WindowsPath('data/dandelion'), WindowsPath('data/roses'), WindowsPath('data/sunflowers'), WindowsPath('data/tulips')]
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [3]:
# 異なる画像サイズはcrop and padしたいので、最大サイズを確認

from PIL import Image
max_width = 0
max_height = 0
image_count = 0

for folder_path in flower_folders:
    for file_path in folder_path.iterdir():
        image = Image.open(file_path)
        size = image.size
        image.close()
        single_path = str(file_path)

        # 画像リストの中から最大幅、高さを取得
        if size[0] > max_width:
            max_width = size[0]
        if size[1] > max_height:
            max_height = size[1]
        image_count += 1
print(f'Max Width: {max_width}\nMax Height: {max_height}')
print(f'Number of Images: {image_count}')

Max Width: 1024
Max Height: 442
Number of Images: 3670


In [64]:
# Features datasetを作成

img_raw = tf.io.read_file(single_path)
img_tensor = tf.image.decode_image(img_raw)
img_tensor = tf.image.resize_with_crop_or_pad(img_tensor, 100, 100)

# Tensorをlist化し、3-dimensionを4-dimensionへ変更
stacked = tf.stack([img_tensor, img_tensor])
print(stacked.shape)
z = tf.concat([stacked, tf.reshape(img_tensor,[1,100,100,3])],axis=0)
print(z.shape)

(2, 100, 100, 3)
(3, 100, 100, 3)


In [66]:
''' 入力の確認用に使用

# CNNの初期化。単純なSequentialモデルで作成。
cnn = tf.keras.models.Sequential()

# 畳み込み https://keras.io/ja/layers/convolutional/
# 128x128 RGB画像ではinput_shape=(128, 128, 3)となります．
# 最初のレイヤーだけは、入り口となる入力シェイプが必要
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[100, 100, 3], data_format='channels_last'))

# Poolingにより、ダウンサンプリング
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# 2層目の中間層を追加
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# 接続
# unitsは   レイヤーの出力形状
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# 出力層
#cnn.add(tf.keras.layers.Dense(units=10, activation='sigmoid'))
cnn.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compiling the CNN
# 評価関数の選定 -> https://keras.io/ja/metrics/
cnn.compile(optimizer = 'adam', loss = tf.keras.losses.sparse_categorical_crossentropy, metrics = ['accuracy'])

y_train = np.array([1,1,1])
answer = cnn.fit(z, y_train, epochs = 1, batch_size = 20)
'''


Train on 3 samples
3/3 [==============================] - 0s 160ms/sample - loss: 48.1106 - accuracy: 0.0000e+00
